In [1]:
import nltk
import sys
import numpy as np
nltk.download('brown')
nltk.download('universal_tagset')
data = nltk.corpus.brown.tagged_sents(tagset='universal')
all_tags = ['#EOS#','#UNK#','ADV', 'NOUN', 'ADP', 'PRON', 'DET', '.', 'PRT', 'VERB', 'X', 'NUM', 'CONJ', 'ADJ']

data = np.array([ [(word.lower(),tag) for word,tag in sentence] for sentence in data ])

[nltk_data] Downloading package brown to
[nltk_data]     C:\Users\Banipreet\AppData\Roaming\nltk_data...
[nltk_data]   Package brown is already up-to-date!
[nltk_data] Downloading package universal_tagset to
[nltk_data]     C:\Users\Banipreet\AppData\Roaming\nltk_data...
[nltk_data]   Package universal_tagset is already up-to-date!


In [2]:
from sklearn.cross_validation import train_test_split
train_data,test_data = train_test_split(data,test_size=0.25,random_state=42)

C:\Users\Banipreet\Anaconda3\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [4]:
from IPython.display import HTML, display
def draw(sentence):
    words,tags = zip(*sentence)
    display(HTML('<table><tr>{tags}</tr>{words}<tr></table>'.format(
                words = '<td>{}</td>'.format('</td><td>'.join(words)),
                tags = '<td>{}</td>'.format('</td><td>'.join(tags)))))
    

draw(data[4])
draw(data[12])

DET,NOUN,VERB,PRON,VERB,VERB,ADP,ADJ,ADP,NOUN,NOUN,CONJ,NOUN,NOUN,.,VERB,ADJ,CONJ,ADJ,CONJ,ADV,ADJ,.,.,the,jury,said,it,did,find,that,many,of,georgia's,registration,and,election,laws,``,are,outmoded,or,inadequate,and,often,ambiguous,'',.


PRON,VERB,ADP,DET,ADJ,NOUN,.,VERB,VERB,NOUN,CONJ,VERB,DET,ADJ,NOUN,ADP,ADP,DET,ADJ,NOUN,ADP,DET,NOUN,VERB,VERB,VERB,.,.,it,urged,that,the,next,legislature,``,provide,enabling,funds,and,re-set,the,effective,date,so,that,an,orderly,implementation,of,the,law,may,be,effected,'',.


In [6]:
from collections import Counter
word_counts = Counter()
for sentence in data:
    words,tags = zip(*sentence)
    word_counts.update(words)

all_words = ['#EOS#','#UNK#']+list(list(zip(*word_counts.most_common(10000)))[0])

In [7]:
from collections import defaultdict
word_to_id = defaultdict(lambda:1,{word:i for i,word in enumerate(all_words)})
tag_to_id = {tag:i for i,tag in enumerate(all_tags)}

In [8]:
def to_matrix(lines,token_to_id,max_len=None,pad=0,dtype='int32',time_major=False):
    """Converts a list of names into rnn-digestable matrix with paddings added after the end"""
    
    max_len = max_len or max(map(len,lines))
    matrix = np.empty([len(lines),max_len],dtype)
    matrix.fill(pad)

    for i in range(len(lines)):
        line_ix = list(map(token_to_id.__getitem__,lines[i]))[:max_len]
        matrix[i,:len(line_ix)] = line_ix

    return matrix.T if time_major else matrix

In [9]:
batch_words,batch_tags = zip(*[zip(*sentence) for sentence in data[-3:]])

print("Word ids:")
print(to_matrix(batch_words,word_to_id))
print("Tag ids:")
print(to_matrix(batch_tags,tag_to_id))

Word ids:
[[   2 3057    5    2 2238 1334 4238 2454    3    6   19   26 1070   69
     8 2088    6    3    1    3  266   65  342    2    1    3    2  315
     1    9   87  216 3322   69 1558    4    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0]
 [  45   12    8  511 8419    6   60 3246   39    2    1    1    3    2
   845    1    3    1    3   10 9910    2    1 3470    9   43    1    1
     3    6    2 1046  385   73 4562    3    9    2    1    1 3250    3
    12   10    2  861 5240   12    8 8936  121    1    4]
 [  33   64   26   12  445    7 7346    9    8 3337    3    1 2811    3
     2  463  572    2    1    1 1649   12    1    4    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0]]
Tag ids:
[[ 6  3  4  6  3  3  9  9  7 12  4  5  9  4  6  3 12  7  9  7  9  8  4  6
   3  7  6 13  3  4  6  3  9  4  3  7  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0

In [10]:
import keras
import keras.layers as L

model = keras.models.Sequential()
model.add(L.InputLayer([None],dtype='int32'))
model.add(L.Embedding(len(all_words),50))
model.add(L.SimpleRNN(64,return_sequences=True))

#add top layer that predicts tag probabilities
stepwise_dense = L.Dense(len(all_tags),activation='softmax')
stepwise_dense = L.TimeDistributed(stepwise_dense)
model.add(stepwise_dense)

C:\Users\Banipreet\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [11]:
from keras.utils.np_utils import to_categorical
BATCH_SIZE=32
def generate_batches(sentences,batch_size=BATCH_SIZE,max_len=None,pad=0):
    assert isinstance(sentences,np.ndarray),"Make sure sentences is q numpy array"
    
    while True:
        indices = np.random.permutation(np.arange(len(sentences)))
        for start in range(0,len(indices)-1,batch_size):
            batch_indices = indices[start:start+batch_size]
            batch_words,batch_tags = [],[]
            for sent in sentences[batch_indices]:
                words,tags = zip(*sent)
                batch_words.append(words)
                batch_tags.append(tags)

            batch_words = to_matrix(batch_words,word_to_id,max_len,pad)
            batch_tags = to_matrix(batch_tags,tag_to_id,max_len,pad)

            batch_tags_1hot = to_categorical(batch_tags,len(all_tags)).reshape(batch_tags.shape+(-1,))
            yield batch_words,batch_tags_1hot
        

In [12]:
def compute_test_accuracy(model):
    test_words,test_tags = zip(*[zip(*sentence) for sentence in test_data])
    test_words,test_tags = to_matrix(test_words,word_to_id),to_matrix(test_tags,tag_to_id)

    #predict tag probabilities of shape [batch,time,n_tags]
    predicted_tag_probabilities = model.predict(test_words,verbose=1)
    predicted_tags = predicted_tag_probabilities.argmax(axis=-1)

    #compute accurary excluding padding
    numerator = np.sum(np.logical_and((predicted_tags == test_tags),(test_words != 0)))
    denominator = np.sum(test_words != 0)
    return float(numerator)/denominator


class EvaluateAccuracy(keras.callbacks.Callback):
    def on_epoch_end(self,epoch,logs=None):
        sys.stdout.flush()
        print("\nMeasuring validation accuracy...")
        acc = compute_test_accuracy(self.model)
        print("\nValidation accuracy: %.5f\n"%acc)
        sys.stdout.flush()
        

In [13]:
model.compile('adam','categorical_crossentropy')

model.fit_generator(generate_batches(train_data),len(train_data)/BATCH_SIZE,
                    callbacks=[EvaluateAccuracy()], epochs=5,)

Epoch 1/5
1344/1343 [==============================] - 77s 57ms/step - loss: 0.2377

Measuring validation accuracy...
14335/14335 [==============================] - 15s 1ms/step

Validation accuracy: 0.93969

Epoch 2/5
1344/1343 [==============================] - 82s 61ms/step - loss: 0.0579

Measuring validation accuracy...
14335/14335 [==============================] - 14s 1ms/step

Validation accuracy: 0.94471

Epoch 3/5
1344/1343 [==============================] - 75s 56ms/step - loss: 0.0510

Measuring validation accuracy...
14335/14335 [==============================] - 15s 1ms/step

Validation accuracy: 0.94619

Epoch 4/5
1344/1343 [==============================] - 72s 53ms/step - loss: 0.0467

Measuring validation accuracy...
14335/14335 [==============================] - 14s 998us/step

Validation accuracy: 0.94688

Epoch 5/5
1344/1343 [==============================] - 70s 52ms/step - loss: 0.0432

Measuring validation accuracy...
14335/14335 [==============================]

In [14]:
acc = compute_test_accuracy(model)
print("Simple RNN accuracy: %.5f"%acc)


14335/14335 [==============================] - 15s 1ms/step
Simple RNN accuracy: 0.94594


In [15]:
#Bidirectional SimpleRNN
model = keras.models.Sequential()

model.add(L.InputLayer([None],dtype='int32'))
model.add(L.Embedding(len(all_words),50))
model.add(L.Bidirectional(L.SimpleRNN(64,return_sequences=True)))

stepwise_dense = L.Dense(len(all_tags),activation='softmax')
stepwise_dense = L.TimeDistributed(stepwise_dense)
model.add(stepwise_dense)

In [16]:
model.compile('adam','categorical_crossentropy')

model.fit_generator(generate_batches(train_data),len(train_data)/BATCH_SIZE,
                    callbacks=[EvaluateAccuracy()], epochs=5,)

Epoch 1/5
1344/1343 [==============================] - 121s 90ms/step - loss: 0.1863

Measuring validation accuracy...
14335/14335 [==============================] - 26s 2ms/step

Validation accuracy: 0.95581

Epoch 2/5
1344/1343 [==============================] - 116s 87ms/step - loss: 0.0423

Measuring validation accuracy...
14335/14335 [==============================] - 25s 2ms/step

Validation accuracy: 0.95967

Epoch 3/5
1344/1343 [==============================] - 73s 54ms/step - loss: 0.0352

Measuring validation accuracy...
14335/14335 [==============================] - 11s 765us/step

Validation accuracy: 0.96254

Epoch 4/5
1344/1343 [==============================] - 52s 39ms/step - loss: 0.0297

Measuring validation accuracy...
14335/14335 [==============================] - 11s 781us/step

Validation accuracy: 0.96152

Epoch 5/5
1344/1343 [==============================] - 52s 39ms/step - loss: 0.0252

Measuring validation accuracy...
14335/14335 [===========================

In [17]:
acc = compute_test_accuracy(model)
print("\nBidirectional RNN accuracy: %.5f"%acc)

14335/14335 [==============================] - 11s 785us/step

Bidirectional RNN accuracy: 0.96125


In [18]:
#Bidirectional LSTM
model = keras.models.Sequential()

model.add(L.InputLayer([None],dtype='int32'))
model.add(L.Embedding(len(all_words),50))
model.add(L.Bidirectional(L.LSTM(64, return_sequences=True)))
#model.add(L.GRU(32,return_sequences=True))
#model.add(L.LSTM(16,return_sequences=True))


stepwise_dense = L.Dense(len(all_tags),activation='softmax')
stepwise_dense = L.TimeDistributed(stepwise_dense)
model.add(stepwise_dense)

In [19]:
model.compile('adam','categorical_crossentropy')

model.fit_generator(generate_batches(train_data),len(train_data)/BATCH_SIZE,
                    callbacks=[EvaluateAccuracy()], epochs=5,)

Epoch 1/5
1344/1343 [==============================] - 178s 132ms/step - loss: 0.2636

Measuring validation accuracy...
14335/14335 [==============================] - 37s 3ms/step

Validation accuracy: 0.95459

Epoch 2/5
1344/1343 [==============================] - 178s 133ms/step - loss: 0.0439

Measuring validation accuracy...
14335/14335 [==============================] - 37s 3ms/step

Validation accuracy: 0.96181

Epoch 3/5
1344/1343 [==============================] - 178s 132ms/step - loss: 0.0363

Measuring validation accuracy...
14335/14335 [==============================] - 41s 3ms/step

Validation accuracy: 0.96364

Epoch 4/5
1344/1343 [==============================] - 192s 143ms/step - loss: 0.0314

Measuring validation accuracy...
14335/14335 [==============================] - 38s 3ms/step

Validation accuracy: 0.96492

Epoch 5/5
1344/1343 [==============================] - 185s 138ms/step - loss: 0.0280

Measuring validation accuracy...
14335/14335 [=======================

In [20]:
acc = compute_test_accuracy(model)
print("\nBidirectional LSTM accuracy: %.5f"%acc)

14335/14335 [==============================] - 38s 3ms/step

Bidirectional LSTM accuracy: 0.96504
